#### Release Report

In [ ]:
Quadrant = 
    VAR _aogStatus = RELATED('Sales Order'[Sales Order Priority]) = "High - 1"
    VAR _airbus = 'Sales Line'[AirbusColumn] = "Airbus"
    VAR _notAirbus = 'Sales Line'[AirbusColumn] <> "Airbus"
    VAR _qaStatus = 'Sales Line'[Not in QA]
    VAR _releaseStatus = IF(_qaStatus >= 'Sales Line'[RemainSalesPhysical], "Released", "Unreleased") = "Released" /*nOT IN QA*/
    VAR _poStatus = RELATED('Sales Order'[Purchase Order?]) = "NO PO"
    VAR NotOneOrTwo = NOT(OR(_aogStatus, _airbus))
    VAR _workStatus = 'Sales Line'[Q3Release] = 1
    VAR _loadId = 'Sales Line'[Q3ReleaseLoadID] = 1
    VAR _okToShip = 'Sales Line'[ICI] = "Ok"
    VAR _confirmedOrder = RELATED('Sales Order'[Confirmation Status]) = "Confirmed"
    VAR _checkMRP = 'Sales Line'[ICI] = "Ok-CheckMRP"
    VAR _workStatusOpenInProgressClosed = 'Sales Line'[WorkStatusOpenInProgressClosed] = "1"
RETURN
    SWITCH(TRUE(),
        _aogStatus, 0,
        AND(_loadId, _workStatusOpenInProgressClosed), 3,
       _airbus, 1,
        AND(AND(_notAirbus,OR(_okToShip, _checkMRP)), _confirmedOrder), 2,
        _poStatus, 4,
        4
    )

In [ ]:
## AirbudD1 Datacrafters

AIRBUSD1 = 
OR(
    AND(
        CONTAINSSTRING('Sales Line'[DeliveryName], "Airbus"), TRUE()
    ),
    RELATED('Sales Order'[Sales Order Airbus Check Last]
    ))



Sales Order Airbus Check Last = 
AND(
    CONTAINSSTRING('Sales Order'[Customer Name], "Airbus"),
    AND(
        NOT( CONTAINSSTRING('Sales Order'[Customer Name], "Airbus Atlantic") ),
        NOT( CONTAINSSTRING( 'Sales Order'[Customer Name], "Airbus SAS") )
    )
)

### Capacity Plan

#### Running Totals

In [ ]:
RunningTotalLoad2 = 
CALCULATE(
    [Total Load],
    FILTER(
        Main, 
        Main[AllStart of week] <= EARLIER(Main[AllStart of week])
    )
)

In [ ]:
RunningCapacity = 
CALCULATE(
    SUM('Weekly Capacity'[Standard Hours/Week Efficiency]),
    FILTER(
        'Weekly Capacity', 
        'Weekly Capacity'[Week] <= EARLIER('Weekly Capacity'[Week])
    )
)

In [ ]:
RunningCapacity of Total = 
CALCULATE(
    SUM('Weekly Capacity'[Total Hours/Week @ Efficiency]),
    FILTER(
        'Weekly Capacity', 
        'Weekly Capacity'[Week] <= EARLIER('Weekly Capacity'[Week])
    )
)

In [ ]:
RunningPlannedLoad = 
CALCULATE(
    SUM('Weekly Capacity'[PlannedLoad]),
    FILTER(
        'Weekly Capacity', 
        'Weekly Capacity'[Week] <= EARLIER('Weekly Capacity'[Week])
    )
)

In [ ]:
RunningProductionLoad = 
CALCULATE(
    SUM('Weekly Capacity'[ProductionLoad]),
    FILTER(
        'Weekly Capacity', 
        'Weekly Capacity'[Week] <= EARLIER('Weekly Capacity'[Week])
    )
)

In [ ]:
RunningTotalLoad = 
CALCULATE(
    SUM('Weekly Capacity'[Total Load]),
    FILTER(
        'Weekly Capacity', 
        'Weekly Capacity'[Week] <= EARLIER('Weekly Capacity'[Week])
    )
)

#### Capacity

In [ ]:
Capacity = 
VAR _minDate = FIRSTDATE(Main[AllStart of week]) + 0

RETURN
CALCULATE(
    SUM('Weekly Capacity'[Standard Hours/Week Efficiency]), 
    ('Calendar'[Date] >= _minDate && 'Calendar'[Date] <= MAX('Weekly Capacity'[Week])), 
    ALL('Weekly Capacity'))


In [ ]:
Capacity of Total = 
VAR _minDate = FIRSTDATE(Main[AllStart of week]) + 0

RETURN
CALCULATE(
    SUM('Weekly Capacity'[Total Hours/Week @ Efficiency]), 
    ('Calendar'[Date] >= _minDate && 'Calendar'[Date] <= MAX('Weekly Capacity'[Week])), 
    ALL('Weekly Capacity'))

#### Burndowns

In [ ]:
TOTAL Burndown of Standard = 
VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Total EXCESS]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningTotalLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningTotalLoad]-'Weekly Capacity'[RunningCapacity]-_cume_offset)
RETURN
_result

In [ ]:
TOTAL Burndown of Total = 
        VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Total EXCESS of Total]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
        VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningTotalLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
        VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningTotalLoad]-'Weekly Capacity'[RunningCapacity of Total]-_cume_offset)
RETURN
        _result

In [ ]:
Planned Burndown of Standard = 
VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Planned EXCESS]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningPlannedLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningPlannedLoad]-'Weekly Capacity'[RunningCapacity]-_cume_offset)
RETURN
IF(_result < 0, 0, _result)

In [ ]:
Planned Burndown of Total = 
        VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Planned EXCESS of Total]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
        VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningPlannedLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
        VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningPlannedLoad]-'Weekly Capacity'[RunningCapacity of Total]-_cume_offset)
RETURN
        IF(_result < BLANK(), 0, _result)

In [ ]:
Production Burndown of Standard = 
VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Production EXCESS]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningProductionLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningProductionLoad]-'Weekly Capacity'[RunningCapacity]-_cume_offset)
RETURN
_result

In [ ]:
Production Burndown of Total = 
        VAR _cume_min = CALCULATE(MIN('Weekly Capacity'[Production EXCESS of Total]), 'Weekly Capacity'[Week] <= MAX('Weekly Capacity'[Week]), ALL('Weekly Capacity'))
        VAR _cume_offset = IF(ISBLANK(SUM('Weekly Capacity'[RunningProductionLoad])), BLANK(), MIN(0, COALESCE(_cume_min,0)))
        VAR _result = SUMX('Weekly Capacity', 'Weekly Capacity'[RunningProductionLoad]-'Weekly Capacity'[RunningCapacity of Total]-_cume_offset)
RETURN
        _result

#### Excesss

In [ ]:
Planned EXCESS = 
        
VAR _result = 'Weekly Capacity'[RunningPlannedLoad] - 'Weekly Capacity'[RunningCapacity]
        
RETURN
        
_result

In [ ]:
Planned EXCESS of Total = 
                
VAR _result = 'Weekly Capacity'[RunningPlannedLoad] - 'Weekly Capacity'[RunningCapacity of Total]
                
RETURN
                
_result

In [ ]:
Production EXCESS = 
        
VAR _result = 'Weekly Capacity'[RunningProductionLoad] - 'Weekly Capacity'[RunningCapacity]
        
RETURN
        
_result

In [ ]:
Production EXCESS of Total = 
                
VAR _result = 'Weekly Capacity'[RunningProductionLoad] - 'Weekly Capacity'[RunningCapacity of Total]
                
RETURN
                
_result

In [ ]:
Total EXCESS = 
        
VAR _result = 'Weekly Capacity'[RunningTotalLoad] - 'Weekly Capacity'[RunningCapacity]
        
RETURN
        
_result

In [ ]:
Total EXCESS of Total = 
VAR _minDate = FIRSTDATE(Main[AllStart of week]) + 0
        
VAR _result = 'Weekly Capacity'[RunningTotalLoad] - 'Weekly Capacity'[RunningCapacity of Total]
        
RETURN
        
_result